# 画像ファイルに対応するキロ程紐づけ用のJSONファイルを作成するノートブック
```
以下のコマンドを順番に実行していく
想定しているディレクトリ構造は以下のとおり
.
├ imgs
│  └ dir_area    :線区単位の画像を保存
│     ├ HD11     :カメラ毎の画像を保存
│     ├ HD12     :以下、同じ
│     ├ HD21         
│     ├ HD21 
│     ├ HD31 
│     └ HD32 
├ TDM
│  └ TDM_GazoFileIndex.xlsx  :車モニから出力したマスターデータ
└ imgKiro.ipynb  :本ファイル
```

## 1. ライブラリを読み込む

In [20]:
import importlib

import gc
import re
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw, ImageFont
from src.config import appProperties
import src.helpers as helpers
import src.visualize as vis
importlib.reload(helpers)
importlib.reload(vis)

config = appProperties('config.yml')

print("準備完了")

準備完了


## 2. 初期設定
```
ファイルパスなどを設定する
車モニのマスターデータは検測キロ程になっているため、NEWSSキロ程に合わせる基準を設定する
```

In [2]:
# 初期設定

# 車モニのマスターデータのファイルパス
tdm_fpath = f'{config.tdm_dir}/TDM_GazoFileIndex.xlsx'
print(f"車モニ マスターデータのファイルパス:{tdm_fpath}")

# 画像ファイルを保存したフォルダ
dir_area = "Takasaki_99_Omiya-Kitaageo_down_20230509_day"
print(f"解析対象フォルダ:{dir_area}")

# 境界部でのNEWSSキロ程
# 例) 1号柱のキロ程を指定する
#    電柱番号: int型
#    キロ程  : float型 km単位
EkiCd_NEWSS = 14010
pole_num_NEWSS = 1
pole_kilo_NEWSS = 3.561
print(f"基準にする電柱：{pole_num_NEWSS}号柱 {pole_kilo_NEWSS}km")

車モニ マスターデータのファイルパス:TDM/TDM_GazoFileIndex.xlsx
解析対象フォルダ:Takasaki_99_Omiya-Kitaageo_down_20230509_day
基準にする電柱：1号柱 3.561km


## 3. 車モニのマスターデータを読込んでいく

In [4]:
%%time
# 車モニのマスタデータを読み込む
print("車モニのマスターデータを読み込みます ※少し時間がかかります")
df_tdm = pd.read_excel(tdm_fpath)
print("車モニのマスターデータを読み込みました")
print(f"データフレームのサイズ:{df_tdm.shape}")
# print("車モニ マスターデータに含まれるカラム名")
# df_tdm.columns

車モニのマスターデータを読み込みます ※少し時間がかかります
車モニのマスターデータを読み込みました
データフレームのサイズ:(53398, 90)
CPU times: user 42.2 s, sys: 122 ms, total: 42.4 s
Wall time: 51.2 s


## 4. データの前処理・NEWSSキロ程に変換する

In [5]:
# 車モニ マスターデータにおける、基準にする電柱の検測キロ程を取得
# オフセット値を設定する
pole_num_matched = df_tdm[df_tdm['EkiCd'] == EkiCd_NEWSS].query(f"DenchuNo == {pole_num_NEWSS}").index.tolist()
pole_kiro_twins = df_tdm.loc[min(pole_num_matched)]['KiroTei']
pole_kiro_offset = pole_kilo_NEWSS - pole_kiro_twins
print(f"基準にする電柱の検測キロ程: {pole_kiro_twins}km")
print(f"　　　キロ程のオフセット値: {pole_kiro_offset}km")


# 欲しい列だけ抽出し、画像ファイル名がNaNの行は削除
df_tdm_temp = df_tdm.filter([
    'SenbetsuCd', 'SokuteiDate', 'DenchuNo', 'KiroTei', 'GazoFileNameHD11', 'GazoFileNameHD12',
    'GazoFileNameHD21', 'GazoFileNameHD22', 'GazoFileNameHD31', 'GazoFileNameHD32'
]).copy()
# ]).dropna(subset=[
#     'GazoFileNameHD11', 'GazoFileNameHD12', 'GazoFileNameHD21', 
#     'GazoFileNameHD22', 'GazoFileNameHD31', 'GazoFileNameHD32'
# ]).copy()

# print("NaNの数をチェック")
# print(df_tdm_temp.isna().sum())
# df_tdm_temp
print("必要な情報だけフィルタリングしました")
print(f"データフレームのサイズ:{df_tdm_temp.shape}")


# SenbetsuCd_dict = {
#     21: "下り線",
#     22: "上り線"
# }
# 測定日・走行条件を表示する
SokuteiDate = df_tdm_temp['SokuteiDate'].unique()
print("読み込まれたデータベースに含まれる測定日↓")
for i, date in enumerate(SokuteiDate):
    # 線別コードを取得
    SenbetsuCd = df_tdm_temp[df_tdm_temp['SokuteiDate'] == date]['SenbetsuCd'].unique().item()
    print(f"{i}> 走行日：{date} 線別：{config.SenbetsuCd[SenbetsuCd]}")
    
# 車モニのデータは検測車キロ程なので、NEWSSキロ程に合わせてオフセットする
df_tdm_temp['KiroTei_NEWSS'] = [KiroTei + pole_kiro_offset for KiroTei in df_tdm_temp['KiroTei']]
print("NEWSSキロ程に合うように、キロ程をオフセットしました")
print()
df_tdm_temp

基準にする電柱の検測キロ程: 1.9815km
　　　キロ程のオフセット値: 1.5795km
必要な情報だけフィルタリングしました
データフレームのサイズ:(53398, 10)
読み込まれたデータベースに含まれる測定日↓
0> 走行日：20220917 線別：下り線
1> 走行日：20221130 線別：下り線
2> 走行日：20230216 線別：下り線
3> 走行日：20230509 線別：下り線
4> 走行日：20230510 線別：上り線
5> 走行日：20230920 線別：下り線
6> 走行日：20230921 線別：上り線
NEWSSキロ程に合うように、キロ程をオフセットしました



,SenbetsuCd,SokuteiDate,DenchuNo,KiroTei,GazoFileNameHD11,GazoFileNameHD12,GazoFileNameHD21,GazoFileNameHD22,GazoFileNameHD31,GazoFileNameHD32,KiroTei_NEWSS
0,21,20220917,901,1.9495,NaN,NaN,NaN,NaN,NaN,NaN,3.5290
1,21,20220917,901,1.9505,2022_0347_HD11_01_00015607,2022_0347_HD12_01_00015607,2022_0347_HD21_01_00015608,2022_0347_HD22_01_00015608,2022_0347_HD31_01_00015607,2022_0347_HD32_01_00015607,3.5300
2,21,20220917,901,1.9515,NaN,NaN,NaN,NaN,NaN,NaN,3.5310
3,21,20220917,901,1.9525,2022_0347_HD11_01_00015608,2022_0347_HD12_01_00015608,2022_0347_HD21_01_00015609,2022_0347_HD22_01_00015609,2022_0347_HD31_01_00015608,2022_0347_HD32_01_00015608,3.5320
4,21,20220917,901,1.9535,NaN,NaN,NaN,NaN,NaN,NaN,3.5330
...,...,...,...,...,...,...,...,...,...,...,...
53393,22,20230921,12,8.5438,2023_0379_HD11_01_00073022,2023_0379_HD12_01_00073022,2023_0379_HD21_01_00073025,2023_0379_HD22_01_00073025,2023_0379_HD31_01_00073024,2023_0379_HD32_01_00073024,10.1233
53394,22,20230921,12,8.5448,NaN,NaN,2023_0379_HD21_01_00073024,2023_0379_HD22_01_00073024,NaN,NaN,10.1243
53395,22,20230921,12,8.5448,NaN,NaN,NaN,NaN,NaN,NaN,10.1243
53396,22,20230921,12,8.5458,2023_0379_HD11_01_00073021,2023_0379_HD12_01_00073021,NaN,NaN,2023_0379_HD31_01_00073023,2023_0379_HD32_01_00073023,10.1253


## 5. 摩耗判定システムで利用可能なJSONファイルを作成する

In [6]:
# 初期設定の情報を出力
print(f"画像フォルダ名：{dir_area}")
print(f"基準にする電柱：{pole_num_NEWSS}")
print(f"　　　　キロ程：{pole_kilo_NEWSS}")
print(f"検測キロ程ズレ：{pole_kiro_offset}")

imgKilo = {}
imgKilo_temp_values = {}
for camera_num in config.camera_types:
    # カメラフォルダ内の画像ファイルを取得
    image_dir = f"{config.image_dir}/{dir_area}/{camera_num}/"
    print(image_dir)
    list_images = helpers.list_images(image_dir)
    print(f"Image counts:{len(list_images)}")

    # 画像ファイル名に対応するキロ程を抽出して辞書型で記録する
    imgKilo_temp = {}
    imgKilo_temp_values = {}
    for fname in tqdm(list_images):
    # for fname in list_images:
        image_name = re.split('[./]', fname)[-2]
        # image_name = "2023_0117_HD11_01_00017734"
        # print(image_name)
        df_tdm_Series = df_tdm_temp[df_tdm_temp[f"GazoFileName{camera_num}"] == image_name].copy()
        if df_tdm_Series.empty:
            continue
        else:
            DenchuNo = df_tdm_Series['DenchuNo'].item()
            KiroTei = df_tdm_Series['KiroTei_NEWSS'].item()
        
        imgKilo_temp_values["DenchuNo"] = DenchuNo
        imgKilo_temp_values["KiroTei"] = KiroTei
        imgKilo_temp[image_name] = imgKilo_temp_values.copy()
        # if image_name == "2023_0117_HD11_01_00017734":
        #     break    # 指定した画像で中断する
        # break
    imgKilo[camera_num] = imgKilo_temp.copy()
    # break

# 結果をJSONファイルに記録する
dir = f"{config.tdm_dir}/{dir_area}.json"
with open(dir, mode="wt", encoding="utf-8") as f:
    json.dump(imgKilo, f, ensure_ascii=False, indent=2, default=helpers.default)

print(f"画像ファイルごとのキロ程情報を{dir}に記録しました")

# print(f"DenchuNo:{DenchuNo} <{type(DenchuNo)}>  KiroTei:{KiroTei} <{type(KiroTei)}>")
# df_tdm_Series

画像フォルダ名：Takasaki_99_Omiya-Kitaageo_down_20230509_day
基準にする電柱：1
　　　　キロ程：3.561
検測キロ程ズレ：1.5795
imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD11/
Image counts:3443


  0%|          | 0/3443 [00:00<?, ?it/s]

imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD12/
Image counts:3443


  0%|          | 0/3443 [00:00<?, ?it/s]

imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD21/
Image counts:3443


...

imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD31/
Image counts:3443


  0%|          | 0/3443 [00:00<?, ?it/s]

imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD32/
Image counts:3443


  0%|          | 0/3443 [00:00<?, ?it/s]

画像ファイルごとのキロ程情報をTDM/Takasaki_99_Omiya-Kitaageo_down_20230509_day.jsonに記録しました


## 6. 作成されたJSONファイルのチェック

In [22]:
# 作業用

In [21]:
with open(f"{config.tdm_dir}/{dir_area}.json", 'r') as file:
    kiro_dict = json.load(file)

In [77]:
fname = list_images[0]
image_name = re.split('[./]', fname)[-2]
image_name

'2023_0117_HD32_01_00015314'

In [81]:
camera_num = config.camera_types[0]
camera_num

'HD11'

In [85]:
# カメラフォルダ内の画像ファイルを取得
image_dir = f"{config.image_dir}/{dir_area}/{camera_num}/"
print(image_dir)
list_images = helpers.list_images(image_dir)

imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD11/


In [86]:
image_match_idx = experimental_get_image_match_idx(list_images, kiro_dict, camera_num)
image_match_idx

[25, 3442]

In [96]:
fname = list_images[image_match_idx[0]]
image_name = re.split('[./]', fname)[-2]
kiro_tei_init_head = kiro_dict[camera_num][image_name]['KiroTei']

fname = list_images[image_match_idx[1]]
image_name = re.split('[./]', fname)[-2]
kiro_tei_init_tail = kiro_dict[camera_num][image_name]['KiroTei']

print(f"kiro_tei_init_head: {kiro_tei_init_head}")
print(f"kiro_tei_init_tail: {kiro_tei_init_tail}")

kiro_tei_init_head: 2.9997999999999996
kiro_tei_init_tail: 9.589559999999999


画像1枚:2m, 1,000px -> 2mm/px -> 0.000002km/px -> 0.002km/idx

In [97]:
2 / config.img_width

0.002

In [78]:
list_images[image_match_idx[0]]

'imgs/Takasaki_99_Omiya-Kitaageo_down_20230509_day/HD32/2023_0117_HD32_01_00015339.jpg'

In [ ]:
list_images[0]

In [75]:
image_name.split(".")[0] in kiro_dict[camera_num].keys()

False

In [ ]:
image_name.split(".")[0]

In [25]:
kiro_dict[camera_num].keys()

dict_keys(['2023_0117_HD32_01_00015339', '2023_0117_HD32_01_00015340', '2023_0117_HD32_01_00015341', '2023_0117_HD32_01_00015342', '2023_0117_HD32_01_00015343', '2023_0117_HD32_01_00015344', '2023_0117_HD32_01_00015345', '2023_0117_HD32_01_00015346', '2023_0117_HD32_01_00015347', '2023_0117_HD32_01_00015348', '2023_0117_HD32_01_00015349', '2023_0117_HD32_01_00015350', '2023_0117_HD32_01_00015351', '2023_0117_HD32_01_00015352', '2023_0117_HD32_01_00015353', '2023_0117_HD32_01_00015354', '2023_0117_HD32_01_00015355', '2023_0117_HD32_01_00015356', '2023_0117_HD32_01_00015357', '2023_0117_HD32_01_00015358', '2023_0117_HD32_01_00015359', '2023_0117_HD32_01_00015360', '2023_0117_HD32_01_00015361', '2023_0117_HD32_01_00015362', '2023_0117_HD32_01_00015363', '2023_0117_HD32_01_00015364', '2023_0117_HD32_01_00015365', '2023_0117_HD32_01_00015366', '2023_0117_HD32_01_00015367', '2023_0117_HD32_01_00015368', '2023_0117_HD32_01_00015369', '2023_0117_HD32_01_00015370', '2023_0117_HD32_01_00015371',

In [32]:
[1.5 + ix / 1000 for ix in config.ix_list]

[1.5,
 1.501,
 1.502,
 1.503,
 1.504,
 1.505,
 1.506,
 1.507,
 1.508,
 1.509,
 1.51,
 1.511,
 1.512,
 1.513,
 1.514,
 1.515,
 1.516,
 1.517,
 1.518,
 1.519,
 1.52,
 1.521,
 1.522,
 1.523,
 1.524,
 1.525,
 1.526,
 1.527,
 1.528,
 1.529,
 1.53,
 1.531,
 1.532,
 1.533,
 1.534,
 1.535,
 1.536,
 1.537,
 1.538,
 1.539,
 1.54,
 1.541,
 1.542,
 1.543,
 1.544,
 1.545,
 1.546,
 1.547,
 1.548,
 1.549,
 1.55,
 1.551,
 1.552,
 1.553,
 1.554,
 1.555,
 1.556,
 1.557,
 1.558,
 1.559,
 1.56,
 1.561,
 1.562,
 1.563,
 1.564,
 1.565,
 1.566,
 1.567,
 1.568,
 1.569,
 1.57,
 1.571,
 1.572,
 1.573,
 1.574,
 1.575,
 1.576,
 1.577,
 1.578,
 1.579,
 1.58,
 1.581,
 1.582,
 1.583,
 1.584,
 1.585,
 1.586,
 1.587,
 1.588,
 1.589,
 1.59,
 1.591,
 1.592,
 1.593,
 1.594,
 1.595,
 1.596,
 1.597,
 1.598,
 1.599,
 1.6,
 1.601,
 1.602,
 1.603,
 1.604,
 1.605,
 1.606,
 1.607,
 1.608,
 1.609,
 1.61,
 1.611,
 1.612,
 1.613,
 1.614,
 1.615,
 1.616,
 1.617,
 1.6179999999999999,
 1.619,
 1.62,
 1.621,
 1.6219999999999999,
 1.62

In [84]:
find_kiro_idx_head = 0
find_kiro_idx_tail = 0
for idx, fname in enumerate(list_images):
    image_name = re.split('[./]', fname)[-2]
    if not find_kiro_idx_head:
        # print(f"{idx}> if")
        if image_name.split(".")[0] in kiro_dict[camera_num].keys():
            find_kiro_idx_head = idx
            continue
    else:
        # print(f"{idx}> else")
        if image_name.split(".")[0] in kiro_dict[camera_num].keys():
            # print("if")
            find_kiro_idx_tail = idx

if find_kiro_idx_head and not find_kiro_idx_tail:
    find_kiro_idx_tail = len(list_images) - 1

print(f"find_kiro_idx_head: {find_kiro_idx_head}")
print(f"find_kiro_idx_tail: {find_kiro_idx_tail}")


find_kiro_idx_head: 0
find_kiro_idx_tail: 0


In [43]:
bool(not find_kiro_idx_head)

False

In [44]:
image_name.split(".")[0]

'2023_0117_HD32_01_00018756'

In [79]:
# 修正後
def experimental_get_image_match_idx(list_images, kiro_dict, camera_num):
    """ ローカルの画像ファイル名リストについて、キロ程情報の辞書に含まれる範囲を取得する
    Args:
        list_images(list): ディレクトリ内のファイル名リスト
        kiro_dict(dict): 車モニのデータベースから作成したキロ程情報の辞書
        camera_num(str): 解析中のカメラ番号 (例)HD11
    Return:
        image_match_idx(list): マッチする範囲のリスト[最初, 最後]
    """
    find_kiro_idx_head = 0
    find_kiro_idx_tail = 0
    kiro_keys = set(kiro_dict[camera_num].keys())  # 辞書のキーをセットに変換

    # find_kiro_idx_headを見つける
    for idx, fname in enumerate(list_images):
        image_name = re.split('[./]', fname)[-2]
        if image_name.split(".")[0] in kiro_keys:
            find_kiro_idx_head = idx
            break

    # find_kiro_idx_tailを見つける
    for idx, fname in reversed(list(enumerate(list_images))):
        image_name = re.split('[./]', fname)[-2]
        if image_name.split(".")[0] in kiro_keys:
            find_kiro_idx_tail = idx
            break

    # find_kiro_idx_headが設定されていない場合の処理
    if not find_kiro_idx_head:
        find_kiro_idx_tail = len(list_images) - 1
    return [find_kiro_idx_head, find_kiro_idx_tail]

In [72]:
%%timeit
image_match_idx = experimental_get_image_match_idx(list_images, kiro_dict)

523 µs ± 86.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [73]:
%%time
image_match_idx = experimental_get_image_match_idx(list_images, kiro_dict)
image_match_idx

CPU times: user 3.51 ms, sys: 0 ns, total: 3.51 ms
Wall time: 5.21 ms


[25, 3442]

In [69]:
image_match_idx

[25, 3442]

In [23]:
# 作業用ここまで

In [7]:
# チェック用
# JSONファイルの内容を読み込んでしてCSVで出力する

import pandas as pd
import json

# JSONファイルのパス
file_path = f'{config.tdm_dir}/{dir_area}.json'

# JSONファイルを読み込む
with open(file_path, 'r') as file:
    data = json.load(file)

# JSONデータからPandas DataFrameを作成
# 行名を階層的なキーから生成し、各カラムに値を設定
rows = []
for parent_key, child_dict in data.items():
    for child_key, attributes in child_dict.items():
        file_name = f"{parent_key}_{child_key}"
        rows.append({'file_name': file_name, 'DenchuNo': attributes['DenchuNo'], 'KiroTei': attributes['KiroTei']})

# データフレームを作成
df_json = pd.DataFrame(rows).set_index('file_name')

# JSONから変換したCSVファイルを作成
df_json.to_csv("TDM/{dir_area}.csv")
df_json

,DenchuNo,KiroTei
file_name,,
HD11_2023_0117_HD11_01_00015339,33,2.99980
HD11_2023_0117_HD11_01_00015340,33,3.00180
HD11_2023_0117_HD11_01_00015341,33,3.00380
HD11_2023_0117_HD11_01_00015342,33,3.00580
HD11_2023_0117_HD11_01_00015343,33,3.00780
...,...,...
HD32_2023_0117_HD32_01_00018752,1,9.58148
HD32_2023_0117_HD32_01_00018753,1,9.58350
HD32_2023_0117_HD32_01_00018754,1,9.58552


In [8]:
print("JSONファイルに含まれる電柱番号")
df_json['DenchuNo'].unique()

JSONファイルに含まれる電柱番号


array([ 33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44, 901,
         1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27, 102,  28,  29,  30,  31,  32,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62, 902])

In [9]:
# 直接JSONファイルの中身を確認
with open(file_path, 'r') as file:
    data = json.load(file)

print("読み込まれたカメラ番号")
for key in data.keys():
    print(f"{key} img_counts:{len(data[key])}")

# 画像ファイル名を指定して中身をチェックする
data['HD11']['2023_0117_HD11_01_00015339']

読み込まれたカメラ番号
HD11 img_counts:3418
HD12 img_counts:3418
HD21 img_counts:3418
HD22 img_counts:3418
HD31 img_counts:3418
HD32 img_counts:3418


{'DenchuNo': 33, 'KiroTei': 2.9997999999999996}

In [12]:
image_name = "2023_0117_HD11_01_00015339.jpg"
image_name.split(".")[0]

'2023_0117_HD11_01_00015339'

In [17]:
data['HD11'][image_name.split(".")[0]]['DenchuNo']

33

In [19]:
data['HD11'][image_name.split(".")[0]]['KiroTei']

2.9997999999999996